In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_clastic"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = 'NO_15-9-19_A'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data.columns = [col.replace('LFP_', '') for col in well_data.columns]
well_data.columns

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.thin_beds import ThinBeds
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric, clay_porosity
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction, den_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging, min_max_line

## Estimate Litholoy

In [ ]:
tb_model = ThinBeds()

vsand, vshale, phit, vsh_lam, vsh_dis, vsand_dis, phit_sand = tb_model.estimate_litho_poro(well_data.NPHI, well_data.RHOB)

In [ ]:
from quick_pp.lithology.thin_beds import vsh_phit_xplot

vsh_phit_xplot(vshale, phit, .27, .17)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH.values, vshale, label='VSHale', color='brown')
plt.plot(well_data.DEPTH.values, vsh_lam, label='VSH_Lam', color='blue')
plt.plot(well_data.DEPTH.values, vsh_dis, label='VSH_Dis', color='green')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSHale, VSH_Lam, and VSH_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH.values, vsand, label='VSand', color='gold')
plt.plot(well_data.DEPTH.values, vsand_dis, label='VSand_Dis', color='orange')

plt.xlabel('Depth')
plt.ylabel('Volume')
plt.ylim(0, 1)
plt.title('Comparison of VSand and VSand_Dis Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH.values, phit, label='PHIT', color='blue')
plt.plot(well_data.DEPTH.values, phit_sand, label='PHIT_Sand', color='orange')

plt.xlabel('Depth')
plt.ylabel('Porosity')
plt.ylim(0, 0.5)
plt.title('Comparison of PHIT and PHIT_Sand Across Depth')
plt.legend()
plt.grid()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(nrows=2, figsize=(20, 3), sharex=True)

ax1.plot(well_data.DEPTH, well_data.NPHI, label='NPHI')
ax1.plot(well_data.DEPTH, nphi_max_line, label='NPHI_line')
ax1.set_ylim(.45, -.15)
ax1.legend()

ax2.plot(well_data.DEPTH, well_data.RHOB, label='RHOB')
ax2.plot(well_data.DEPTH, rhob_max_line, label='RHOB_line')
ax2.set_ylim(1.85, 2.85)
ax2.legend()

In [ ]:
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], dry_min1_point=args['dry_sand_point'], **args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, dry_min1_point=args['dry_sand_point'], **args)

In [ ]:
from quick_pp.rock_type import estimate_vsh_gr
vsh_gr_1 = estimate_vsh_gr(well_data['GR'], alpha=0.1)
vsh_gr_2 = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=140)

fig, axs = plt.subplots(2, 1, figsize=(20, 3), sharex=True)

axs[0].plot(well_data.DEPTH, vsh_gr_1, label='vsh_gr_1')
axs[0].plot(well_data.DEPTH, vsh_gr_2, label='vsh_gr_2')
axs[0].legend()

axs[1].plot(well_data.DEPTH, well_data['GR'], label='GR')
axs[1].legend()

In [ ]:
# Specify depth of interest
min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()

In [ ]:
# Density correction based on VSH_GR

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, rhob_corr, label='rhob_corr')
plt.plot(well_data.DEPTH, well_data['RHOB'], label='RHOB')
plt.plot(well_data.DEPTH, rhobhc, label='RHOB_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, well_data['NPHI'], label='NPHI')
plt.plot(well_data.DEPTH, nphihc, label='NPHI_HC')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, badhole_flag, label='badhole_flag')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)

## Estimate Porosity

In [ ]:
args

In [ ]:
# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhob_corr, model='ssc',
    dry_min1_point=args['dry_sand_point'],
    dry_silt_point=args['dry_silt_point'],
    dry_clay_point=args['dry_clay_point'],
)

rho_ma = rho_matrix(vsand, vsilt, vcld)
phid = density_porosity(rhob_corr, rho_ma)

# Normalize lithology
vsand, vsilt, vcld = normalize_volumetric(vsand, vsilt, vcld, phit)

# Calculate vclb: volume of clay bound water and phie
clay_phit = clay_porosity(rhob_corr, args['dry_clay_point'][1])
vclb = vcld * clay_phit
vclw = vcld + vclb

phie = phit - vclb

# Estimate rock types
rock_flag = rock_typing(vclw, higher_is_better=False)

In [ ]:
phit = neu_den_xplot_poro(
    nphihc, rhob_corr, model='ssc',
    dry_min1_point=args['dry_sand_point'],
    dry_silt_point=args['dry_silt_point'],
    dry_clay_point=args['dry_clay_point'],
)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, phit, label='PHIT')
plt.plot(well_data.DEPTH, phid, label='PHID')
plt.plot(well_data.DEPTH, phie, label='PHIE')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlim(min_depth, max_depth)
plt.ylim(0, 0.5)

## Estimate Permeability

In [ ]:
import matplotlib.ticker as ticker

# Estimate permeability
perm = choo_permeability(vclw, vsilt, phit,  B=6, A=5e6)
Swirr = np.where(
    rock_flag == 1, 0.005, np.where(
        rock_flag == 2, 0.2, np.where(
            rock_flag == 3, 0.3, 1
        )
    )
)
perm_kc = kozeny_carman_permeability(phit, S=.5)
perm_t = timur_permeability(phit, Swirr)
perm_c = coates_permeability(phit, Swirr, a=.5)
perm_tx = tixier_permeability(phit, Swirr)

# Plotting
fig, ax = plt.subplots(figsize=(20, 3))
ax.plot(well_data.DEPTH, perm, label='Choo')
ax.plot(well_data.DEPTH, perm_kc, label='KC')
ax.plot(well_data.DEPTH, perm_t, label='Timur')
ax.plot(well_data.DEPTH, perm_c, label='Coates')
ax.plot(well_data.DEPTH, perm_tx, label='Tixier')
ax.set_frame_on(False)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e5)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(
    lambda x, pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(x), 0)))).format(x)))
ax.set_xlim(min_depth, max_depth)
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

## Estimate Water Saturation

In [ ]:
# Debug water saturation
water_salinity = 15000
m = args['sw_m']

temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')
rw_archie = estimate_rw_archie(phit, well_data['RT'], m=m)
rw_shale = estimate_rw_from_shale_trend(well_data['RT'], phit, m=1.2)
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

b_archie = estimate_b_waxman_smits(temp_grad, rw_archie)
b = estimate_b_waxman_smits(temp_grad, rw)

qv = estimate_qv(vcld, phit, cec_clay=.1)

swt_sal = waxman_smits_saturation(well_data['RT'], rw, phit, qv, m=m)
swt_ws_pickett = waxman_smits_saturation(well_data['RT'], .3, phit, qv)
swt = waxman_smits_saturation(well_data['RT'], rw_shale, phit, qv)

# Plotting
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(20, 7))

ax1.plot(well_data.DEPTH, qv, label='Qv')
ax1.set_ylim(0, 10)
ax1.legend()

ax2.plot(well_data.DEPTH, b, label='b')
ax2.plot(well_data.DEPTH, b_archie, label='b_archie')
ax2.legend()

ax3.plot(well_data.DEPTH, well_data['RT'], label='RT')
ax3.plot(well_data.DEPTH, rw, label='rw')
ax3.plot(well_data.DEPTH, rw_archie, label='rw_archie')
ax3.plot(well_data.DEPTH, rw_shale, label='rw_shale')
ax3.set_yscale('log')
ax3.legend()

ax4.plot(well_data.DEPTH, swt_sal, label='SWT_salinity')
ax4.plot(well_data.DEPTH, swt, label='SWT')
ax4.plot(well_data.DEPTH, swt_ws_pickett, label='SWT_ws_pickett')
ax4.legend()

ax4.set_xlim(min_depth, max_depth)
fig.tight_layout()

In [ ]:
from quick_pp.saturation import pickett_plot

# Pickett plot
water_zone = well_data.copy()
water_zone['PHIT'] = phit
water_zone = water_zone[(water_zone.DEPTH > 3995) & (water_zone.DEPTH < 4010)]
fig = pickett_plot(water_zone.RT, water_zone.PHIT, m=1.85, min_rw=.1)

## QAQC Result

In [ ]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['RHOB_CORR'] = rhob_corr
well_data['HC_FLAG'] = hc_flag
well_data['VSAND'] = vsand
well_data['VSILT'] = vsilt
well_data['VCLB'] = vclb
well_data['VCLD'] = vcld
well_data['VCLW'] = vclw
well_data['PHIT'] = phit
well_data['PHIE'] = phie
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['B'] = b
well_data['Qv'] = qv
well_data['M'] = args['sw_m']
well_data['SWT'] = swt
# well_data['SWE'] = swe
well_data['BVW'] = swt * phie
well_data['PERM'] = perm
# well_data['CPERM'] = perm
well_data['ROCK_FLAG'] = rock_flag
well_data['VHC'] = phit * (1 - swt)
well_data['BADHOLE'] = badhole_flag
well_data['VSH_GR'] = vsh_gr

In [ ]:
# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])

In [ ]:
from quick_pp.qaqc import quick_qc

test, summary_df, dist_fig, depth_fig = quick_qc(well_data, return_fig=True)

dist_fig.show()
depth_fig.show()

## Final Result

In [ ]:
# Plot the results
well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
# # Save the results to the well
# well.update_data(well_data)
# well.update_ressum(ressum_df)
# project.save_well(well)

In [ ]:
# project.save()